# 0.0 Imports

In [342]:
import pandas as pd
import regex as re
import numpy as np

from sklearn  import model_selection as ms

# 0.1 Helper functions

# 0.2 Loading data

In [2]:
df_raw_1 = pd.read_csv('../data/data-1703806461.csv')
df_raw_2 = pd.read_csv('../data/data-1703806894.csv')
df_raw_3 = pd.read_csv('../data/data-1703805541.csv')
df_raw_4 = pd.read_csv('../data/data-1703779777.csv')
df_raw_5 = pd.read_csv('../data/data-1703969930.csv')

df_raw = pd.concat([df_raw_1, df_raw_2, df_raw_3, df_raw_4, df_raw_5])
df_raw = df_raw.drop_duplicates(subset=['id'])

# 1.0 Data description

In [295]:
df1 = df_raw.copy()

## 1.1 Data dimension

In [255]:
print('Number of rows: {}'.format(df1.shape[0]))
print('Number of cols: {}'.format(df1.shape[1]))

Number of rows: 22341
Number of cols: 12


## 1.2 Data types

In [256]:
df1.dtypes

Unnamed: 0                    int64
id                            int64
usableAreas                  object
createdAt                    object
displayAddressGeolocation    object
parkingSpaces                object
suites                       object
bathrooms                    object
bedrooms                     object
pricingInfos                 object
amenities                    object
unitFloor                     int64
dtype: object

## 1.3 Check NA

In [257]:
df1.isna().sum()

Unnamed: 0                      0
id                              0
usableAreas                     0
createdAt                       0
displayAddressGeolocation    1218
parkingSpaces                   0
suites                          0
bathrooms                       0
bedrooms                        0
pricingInfos                    0
amenities                       0
unitFloor                       0
dtype: int64

## 1.4 Drop NA

In [296]:
df1 = df1.dropna()

In [297]:
df1 = df1.drop(columns=['Unnamed: 0'], axis=1)

## 1.5 Change types

In [298]:
# Removing the [''] from values, extracting only the numbers
df1['usableAreas'] = df1['usableAreas'].str.extract(r"(\d+)")
df1['parkingSpaces'] = df1['parkingSpaces'].str.extract(r"(\d+)")
df1['suites'] = df1['suites'].str.extract(r"(\d+)")
df1['bathrooms'] = df1['bathrooms'].str.extract(r"(\d+)").astype('int64')
df1['bedrooms'] = df1['bedrooms'].str.extract(r"(\d+)").astype('int64')

# Converting to int64
df1['usableAreas'] = df1['usableAreas'].astype('int64')
df1['parkingSpaces'] = df1['parkingSpaces'].fillna(0).astype('int64')
df1['suites'] = df1['suites'].fillna(0).astype('int64')

In [301]:
# Converting to date type
df1['createdAt_dt'] = pd.to_datetime(df1['createdAt'], format='ISO8601')

In [302]:
df1.dtypes

id                                         int64
usableAreas                                int64
createdAt                                 object
displayAddressGeolocation                 object
parkingSpaces                              int64
suites                                     int64
bathrooms                                  int64
bedrooms                                   int64
pricingInfos                              object
amenities                                 object
unitFloor                                  int64
createdAt_dt                 datetime64[ns, UTC]
dtype: object

## 1.6 Descriptive statistical

In [306]:
num_attributes = df1[['id', 'usableAreas', 'parkingSpaces', 'suites', 'bathrooms', 'bedrooms', 'unitFloor']]
#cat_attributes = df1[[]]

In [308]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [309]:
# Numerical attributes
df_aux = num_attributes.describe().T[['min', 'max', 'mean', 'std']]
range_ = df_aux['min'] - df_aux['max']
kurt = num_attributes.kurt(axis=0)
skew = num_attributes.skew(axis=0)

df_aux['range'] = range_
df_aux['kurt'] = kurt
df_aux['skew'] = skew

df_aux

,min,max,mean,std,range,kurt,skew
id,39560479.000,2679904494.000,2627133188.587,213302692.640,-2640344015.000,118.208,-10.558
usableAreas,10.000,5000.000,97.786,99.832,-4990.000,806.516,19.907
parkingSpaces,0.000,190.000,1.519,1.867,-190.000,5337.881,57.873
suites,0.000,6.000,1.010,1.054,-6.000,0.638,1.094
bathrooms,0.000,12.000,2.242,1.319,-12.000,1.153,1.167
bedrooms,1.000,12.000,2.385,0.836,-11.000,1.464,0.361
unitFloor,0.000,191.000,3.256,7.482,-191.000,62.665,5.673


# 2.0 Feature engineering

In [357]:
df2 = df1.copy()

## 2.1 Feature engineering

In [358]:
# Dealing with 'displayAddressGeolocation'
df2[['locationLon', 'locationLat']] = df2['displayAddressGeolocation'].str.split(',', expand=True)

df2['locationLat'] = df2['locationLat'].str.replace("'lat': ", "").str.replace('}', '').astype(float)

df2['locationLon'] = df2['locationLon'].str.replace("{'lon': ", "").str.replace('{', '').astype(float)

In [359]:
# Dealing with 'pricingInfos'
df2['yearlyIptu'] = df2['pricingInfos'].str.findall(r"(yearlyIptu)': '(\d+)").astype(str).str.extract(r"(\d+)")

df2['price'] = df2['pricingInfos'].str.findall(r"(price)': '(\d+)").astype(str).str.findall(r"(\d+)").apply(lambda x: x[1] if len(x)>1 else x[0])

df2['monthlyCondoFee'] = df2['pricingInfos'].str.findall(r"(monthlyCondoFee)': '(\d+)").astype(str).str.extract(r"(\d+)")

In [364]:
# Converting to int type

df2['price'] = df2['price'].astype('int64')

In [360]:
# Dealing with 'amenities'

## Converting to a list
df2['amenitiesList'] = df2['amenities'].str.findall(r"[a-zA-Z]+")

## Count of amenities
df2['amenitiesCount'] = df2['amenitiesList'].apply(lambda x: len(x))

## Has elevator
df2['amenitiesElevator'] = df2['amenitiesList'].apply(lambda x: 1 if 'ELEVATOR' in x else 0)

## Has pool
df2['amenitiesPool'] = df2['amenitiesList'].apply(lambda x: 1 if 'POOL' in x else 0)

## Has gym
df2['amenitiesGym'] = df2['amenitiesList'].apply(lambda x: 1 if 'GYM' in x else 0)

## Has barbecue
df2['amenitiesBarbecue'] = df2['amenitiesList'].apply(lambda x: 1 if 'BARBECUE' in x else 0)

In [361]:
# Dealing with 'createdAt_dt'

## year
df2['createdAt_year'] = df2['createdAt_dt'].dt.year

## month
df2['createdAt_month'] = df2['createdAt_dt'].dt.month

## day
df2['createdAt_day'] = df2['createdAt_dt'].dt.day

In [365]:
# Creating unitary value - this feature will be the response of our project

df2['unitaryValue'] = df2['price'] / df2['usableAreas']

In [366]:
# Drop equivalents columns
columns = ['createdAt', 'displayAddressGeolocation', 'pricingInfos', 'amenities', 'price']
df2 = df2.drop(columns=columns, axis=1).copy()

## 2.2 Splitting dataset

In [375]:
X = df2.drop('unitaryValue', axis=1).copy()
y = df2['unitaryValue'].copy()

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size = 0.20, random_state = 42)

df2 = pd.concat([X_train, y_train], axis=1)

df_test = pd.concat([X_test, y_test], axis=1)

# 3.0 Feature filtering

In [376]:
df3 = df2.copy()

## 3.1 Row filtering

## 3.2 Column filtering

In [384]:
cols_drop = ['amenitiesList']
df3 = df3.drop(cols_drop, axis=1)
df3.columns

Index(['id', 'usableAreas', 'parkingSpaces', 'suites', 'bathrooms', 'bedrooms',
       'unitFloor', 'createdAt_dt', 'locationLon', 'locationLat', 'yearlyIptu',
       'monthlyCondoFee', 'amenitiesCount', 'amenitiesElevator',
       'amenitiesPool', 'amenitiesGym', 'amenitiesBarbecue', 'createdAt_year',
       'createdAt_month', 'createdAt_day', 'unitaryValue'],
      dtype='object')